# High-level Keras (CNTK) Example

In [29]:
# Parameters
EPOCHS = 10
N_CLASSES=10
BATCHSIZE = 64
LR = 0.01
MOMENTUM = 0.9
GPU = True

LOGGER_URL='msdlvm.southcentralus.cloudapp.azure.com'
LOGGER_USRENAME='admin'
LOGGER_PASSWORD='password'
LOGGER_DB='gpudata'
LOGGER_SERIES='gpu'

In [30]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "cntk"
import keras as K
import cntk
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from os import path

from utils import cifar_for_library, yield_mb, create_logger, Timer
from nb_logging import NotebookLogger, output_to, error_to
from gpumon import db_log_context
import codecs

from influxdb import InfluxDBClient

In [ ]:
client = InfluxDBClient(LOGGER_URL, 8086, LOGGER_USRENAME, LOGGER_PASSWORD, LOGGER_DB)

In [ ]:
node_id = os.getenv('AZ_BATCH_NODE_ID', default='node')
task_id = os.getenv('AZ_BATCH_TASK_ID', default='keras_cntk')
job_id = os.getenv('AZ_BATCH_JOB_ID', default='keras_cntk')

In [ ]:
logger = create_logger(client, node_id=node_id, task_id=task_id, job_id=job_id)

In [31]:
nb_teminal_logger = NotebookLogger(sys.stdout.session, sys.stdout.pub_thread, sys.stdout.name, sys.__stdout__)

In [32]:
rst_out = output_to(nb_teminal_logger)
rst_err = error_to(nb_teminal_logger)

In [33]:
# channels_first is faster
K.backend.set_image_data_format('channels_first')

In [34]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("CNTK: ", cntk.__version__)
print(K.backend.backend())
# Check that channels_last is selected (otherwise slow)
print(K.backend.image_data_format())

OS:  linux
Python:  3.5.4 |Anaconda, Inc.| (default, Oct  5 2017, 08:00:22) 
[GCC 7.2.0]
Keras:  2.0.8
Numpy:  1.13.3
CNTK:  2.1
cntk
channels_first


In [35]:
data_path = path.join(os.getenv('AZ_BATCHAI_INPUT_DATASET'), 'cifar-10-batches-py')

In [36]:
def create_symbol():
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(3, 32, 32)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(N_CLASSES, activation='softmax'))
    return model

In [37]:
def init_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(LR, MOMENTUM),
        metrics = ['accuracy'])
    return m

In [38]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(data_path, channel_first=True, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
Done.
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 692 ms, sys: 676 ms, total: 1.37 s
Wall time: 1.36 s


In [39]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 84 ms, sys: 4 ms, total: 88 ms
Wall time: 85.8 ms


In [40]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 6.37 ms


In [41]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 50, 32, 32)        1400      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 50, 32, 32)        22550     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 50, 16, 16)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 50, 16, 16)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 100, 16, 16)       45100     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 100, 16, 16)       90100     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 100, 8, 8)         0         
__________

In [42]:
with Timer() as t:
    with db_log_context(LOGGER_URL, '8086', LOGGER_USRENAME, LOGGER_PASSWORD, LOGGER_DB, LOGGER_SERIES, 
                    node_id=node_id, task_id=task_id, job_id=job_id):
        # Train model
        model.fit(x_train,
                  y_train,
                  batch_size=BATCHSIZE,
                  epochs=EPOCHS,
                  verbose=1)
print('Training took %.03f sec.' % t.interval)
logger('training duration', value=t.interval)

Epoch 1/10
50000/50000 [==============================] - 24s - loss: 1.8511 - acc: 0.3258    
Epoch 2/10
50000/50000 [==============================] - 21s - loss: 1.3849 - acc: 0.4997    
Epoch 3/10
50000/50000 [==============================] - 19s - loss: 1.1405 - acc: 0.5921    
Epoch 4/10
50000/50000 [==============================] - 20s - loss: 0.9824 - acc: 0.6535    
Epoch 5/10
50000/50000 [==============================] - 20s - loss: 0.8720 - acc: 0.6936    
Epoch 6/10
50000/50000 [==============================] - 21s - loss: 0.7814 - acc: 0.7260    
Epoch 7/10
50000/50000 [==============================] - 25s - loss: 0.7087 - acc: 0.7510    
Epoch 8/10
50000/50000 [==============================] - 24s - loss: 0.6446 - acc: 0.7740    
Epoch 9/10
50000/50000 [==============================] - 26s - loss: 0.5946 - acc: 0.7913    
Epoch 10/10
50000/50000 [==============================] - 25s - loss: 0.5451 - acc: 0.8077    
CPU times: user 1min 13s, sys: 1min 5s, total: 2m

In [43]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 412 ms, sys: 468 ms, total: 880 ms
Wall time: 967 ms


In [44]:
acc=sum(y_guess == y_truth)/len(y_guess)
print("Accuracy: ", acc)
logger('accuracy', value=acc)

Accuracy:  0.7835
